In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib
from datetime import datetime, timedelta

from matplotlib import font_manager, rc, cm
%matplotlib inline

import platform
your_os = platform.system()
if your_os == 'Linux':
    rc('font', family='NanumGothic')
elif your_os == 'Windows':
    ttf = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=ttf).get_name()
    rc('font', family=font_name)
elif your_os == 'Darwin':
    rc('font', family='AppleGothic')
rc('axes', unicode_minus=False)
from sklearn.model_selection import train_test_split
import glob
import os
import time
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean

from tqdm.notebook import tqdm
from pycaret.regression import *
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Pycaret을 이용한 튜닝 및 예측
 - 3 표준편차 기준 이상치 대체 반응변수
 - 왜도 3 이상 값들 로그변환 설명변수
 - VIF < 11 인 값들
 - 부스팅 / 배깅 / 블랜딩 3개 혼합모델

#### 0. 데이터 불러오기
- stepwise로 선택된 변수
- 이상치가 대체된 배출량 data

In [2]:
data = pd.read_csv("행정동1 군집3번의 data 이상치 3.csv")
variable_dummy = np.load("행정동1 label 3 VIF10 선택변수.npy")
print(len(variable_dummy))
data.shape

29


(15324, 323)

In [3]:
variable_dummy

array(['월_일별배출량비율', '연_요일별배출량비율', '일강수량', '고용률', '내국인_목적별 0 레저스포츠', '지방세',
       '연휴전날', 'CO', '요일주점및주류판매결제건수', '일별예상승객총합', 'SO2', '평균 기온',
       '내국인_목적별 0 기타방문', '월_4', '연휴다음날', '연_월농축수산물결제건수비율',
       '연_월주점및주류판매결제건수비율', '연_월주점및주류판매결제금액비율', '분기_3', '장기2일편차',
       '단기1방문편차', '장기3방문편차', '월_7', '월_10', '일강수량월편차', '월_일별결제건수비율',
       'NO2', '장기3거주표준편차', '장기1방문표준편차'], dtype='<U16')

성능평가 방법
- RMSLE

In [6]:
from sklearn.metrics import mean_squared_error,mean_absolute_error

def rmsle(y,pred):
    log_y = np.log1p(y)
    log_pred = np.log1p(pred)
    squared_error = (log_y - log_pred) ** 2
    rmsle = np.sqrt(np.mean(squared_error))
    return rmsle

def rmse(y,pred):
    return np.sqrt(mean_squared_error(y,pred))

def eval_reg(y,pred):
    rmsle_val = rmsle(y,pred)
    rmse_val = rmse(y,pred)
    mae_val = mean_absolute_error(y,pred)
    print("RMSLE : {0:.3f}, RMSE : {1:.3f}, MAE : {2:,.3f}".format(rmsle_val,rmse_val,mae_val))
    return rmsle_val,rmse_val,mae_val

#### 1. 데이터 생성하기
- 로그변환

In [7]:
Y = data["배출량"]
    
dummy = pd.get_dummies(data[["년","월","일","요일","분기"]],columns = ["년","월","일","요일","분기"]).columns
variable_nodummy = list(set(variable_dummy).difference(dummy))
# stepwise로 선택된 변수 중 dummy가 아닌 것들
    
select_dummy = list(set(variable_dummy).difference(variable_nodummy))
# stepwsie로 선택된 변수 중 dummy인 변수들
    
X_1 = pd.get_dummies(data.drop(["일자","행정시","배출량","행정동명"],axis=1)
                         ,columns=["년","월","일","요일","분기"])[variable_nodummy]
# X_1 : 더미변수를 제하고 stepwise로 선택된 모든 변수 dataframe

X_2 = pd.get_dummies(data.drop(["일자","행정시","배출량","행정동명"],axis=1)
                         ,columns=["년","월","일","요일","분기"])[select_dummy]
# X_2 : stepwise로 선택된 모든 더미변수 dataframe

loglist=[]
# 왜도 3인 설명변수 log1p 변환
for val in list(X_1.columns):
    if abs(X_1[val].skew())>3:
        X_1[val] = np.log1p(X_1[val])
        loglist.append(val)
        
X_1 = pd.DataFrame(X_1)
    
X_3 = data["행정동명"]
# Categorical Features인 행정동명
    
X = pd.concat([X_1,X_2,X_3],axis=1)
# X1, X2, X3 데이터 concat

x_train, x_test, y_train, y_test = train_test_split(X, Y,test_size=0.2, shuffle=True, random_state=0)
train = pd.concat([x_train, y_train], axis = 1)

reg = setup(data=train,
                target='배출량',
                numeric_imputation = 'mean',
                normalize = True,
                normalize_method = 'zscore',
                categorical_features = ["행정동명"],
                silent= True,
                fold_strategy = 'timeseries',
                fold = 3,
                session_id = 0,
                verbose=False)

단일 모델 선택하기 위한 성능 비교

In [8]:
best_5_d = compare_models(sort = 'RMSLE', n_select = 5)
print(best_5_d)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,89940.7484,16742737743.0929,128272.1573,0.8435,0.2203,0.1642,2.4833
lightgbm,Light Gradient Boosting Machine,102868.4796,20344831437.0047,141636.5872,0.8098,0.2552,0.1971,1.9833
rf,Random Forest Regressor,105064.0525,22642550645.2385,149291.2350,0.7883,0.2666,0.2069,4.0100
knn,K Neighbors Regressor,134503.4036,36441670314.6667,189457.4219,0.6594,0.3351,0.2804,0.6367
gbr,Gradient Boosting Regressor,140517.9404,36587484087.3491,191062.8682,0.6574,0.3373,0.2843,3.8067
dt,Decision Tree Regressor,145960.4748,46799122226.6701,215178.4859,0.5609,0.3539,0.2620,0.4333
br,Bayesian Ridge,181351.7380,58890434742.3154,242505.0579,0.4491,0.4198,0.3780,0.0567
ridge,Ridge Regression,181281.1979,58916207274.6667,242550.9062,0.4489,0.4201,0.3775,10.8000
llar,Lasso Least Angle Regression,181250.1953,58928973287.0109,242575.5474,0.4488,0.4201,0.3774,0.0333
lasso,Lasso Regression,181278.2031,58930338474.6667,242577.9531,0.4488,0.4202,0.3774,10.2000


[ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=0, verbose=0, warm_start=False), LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=0, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0), RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse'

#### (2)  모델 생성 및 튜닝 (단일 모델 3개 / 배깅 모델 3개 / 부스팅 3개 / 블렌딩 3개)
- RMSLE을 기준으로 상위 모델 선정 
- 단 다른 5개의 성능 지표 비교해서 결정

In [9]:
tuned_models = []

In [ ]:
### 상위 모델 1번 : Extra Tree Regressor
model_01 = create_model("et")

model_01 = tune_model(model_01, fold= 3, n_iter = 100, optimize = 'RMSLE', verbose = True, choose_better = True)
tuned_models.append(model_01)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,104281.3934,23103749416.3813,151999.1757,0.7894,0.2730,0.2055
1,85174.5692,14343421922.3071,119764.0260,0.8564,0.2010,0.1491
2,80366.2825,12781041890.5902,113053.2701,0.8848,0.1870,0.1380
Mean,89940.7484,16742737743.0929,128272.1573,0.8435,0.2203,0.1642
SD,10328.6166,4542914627.6857,16999.7473,0.0400,0.0377,0.0296


IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,13:06:15
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Extra Trees Regressor


,MAE,MSE,RMSE,R2,RMSLE,MAPE


Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


In [ ]:
### 상위 모델 2번 : Random Forest Regressor
model_02 = create_model("rf")

model_02 = tune_model(model_02, fold= 3, n_iter = 100, optimize = 'RMSLE', verbose = True, choose_better = True)
tuned_models.append(model_02)

In [ ]:
#  Light Gradient Boosting Machine 튜닝 (상위 모델 3번)
model_03 = create_model('lightgbm')

model_03 = tune_model(model_03, fold= 3, n_iter = 100, optimize = 'RMSLE', verbose = True, choose_better = True)
tuned_models.append(model_03)

배깅

In [ ]:
prediction_models = []

bag_01_15 = ensemble_model(model_01 , n_estimators = 100, fold=3, optimize = 'RMSLE')
prediction_models.append(bag_01_15)

bag_02_15 = ensemble_model(model_02, n_estimators = 100, fold=3, optimize = 'RMSLE')
prediction_models.append(bag_02_15 )

bag_03_15 = ensemble_model(model_03, n_estimators = 100, fold=3, optimize = 'RMSLE')
prediction_models.append(bag_03_15)

부스팅

In [ ]:
boo_01 = ensemble_model(model_01, method = 'Boosting', fold=3, optimize='RMSLE')
prediction_models.append(boo_01)

In [ ]:
boo_02 = ensemble_model(model_02, method = 'Boosting', fold=3, optimize='RMSLE')
prediction_models.append(boo_02)

In [ ]:
boo_03 = ensemble_model(model_03, method = 'Boosting', fold=3, optimize='RMSLE')
prediction_models.append(boo_03)

블렌딩

In [ ]:
blend_3_soft = blend_models(estimator_list=[model_01, model_02, model_03], fold=3, optimize='RMSLE')
prediction_models.append(blend_3_soft)

#### 생성된 모델에 대한 Train 데이터에 대한 성과 확인

In [ ]:
for model in tuned_models:
    print(model.__class__.__name__)
    display(predict_model(model))

In [ ]:
for model in prediction_models:
    print(model.__class__.__name__)
    display(predict_model(model))

#### 최종 모델 결정 및 성능 평가(test) 
- 상위 모델 3개(일반적으로 단일 모델 1개 / 앙상블 모델 2개해서 적절하게 선택)
- 선택된 최종 모델 3개에 대한 Test 성능 평가
- 이후 3개의 모델에 대한 앙상블(평균)에 대해서 test 성능 평가 후 모델 결정

In [ ]:
final_01 = finalize_model(model_03) # 성능 상위모델 1번: 단일모델 gbr
final_02 = finalize_model(boo_02) # 성능 상위모델 2번: 부스팅 gbr
final_03  = finalize_model(boo_03) # 성능 상위모델 3번: 부스팅 lgbm

In [ ]:
from sklearn.metrics import r2_score
value_list = [] # test를 통해 상위 3개의 모델에 대한 mse / r2값 확인 
for model in [final_01, final_02, final_03]:
    prediction_i = predict_model(model, data = x_test)
    y_pred_i = prediction_i['Label']
    
    value_list.append(y_pred_i)
    
    y = y_test
    rmsle_score = rmsle(y, y_pred_i)
    r2score = r2_score(y, y_pred_i)
    print("rmsle 값: {} // r2_score: {}".format(rmsle_score, r2score))


In [ ]:
# 피쳐 중요도 확인
plot_model(, 'feature_all' )

# 피쳐 중요도 확인
plot_model(,"residuals" )

# q-q-plot
plot_model(,"residuals_interactive")